In [118]:
import pandas as pd
import mysql.connector
import pandas as pd

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

In [119]:
df = pd.read_csv("ttrain.csv", encoding="UTF-8")
df = df.iloc[:, 1:]
df.columns = ["dept", "arr", "category", "cases"]
df[["dept", "arr"]] = df[["dept", "arr"]].apply(lambda x: x.astype(str))
df[["dept", "arr"]] = df[["dept", "arr"]].apply(lambda x: x.str[:5])

code = pd.read_excel("code.xlsx", sheet_name=0)
code.columns = ["code"]
code['code'] = code['code'].apply(lambda x: x[11:])
code = code['code'].str.split(' | ', expand=True)
code = code.drop(code.columns[[1, 3]], axis=1)
code = code.rename(columns={0 : 'code',2 : 'name'})
code['name'] = code['name'].astype(str)

df_merged = pd.merge(df, code, left_on="dept", right_on="code", how="left")
df_merged.rename(columns={"name": "dept_name"}, inplace=True)
df_merged = pd.merge(df_merged, code, left_on="arr", right_on="code", how="left")
df_merged.rename(columns={"name": "arr_name"}, inplace=True)
df = df_merged[["dept", "dept_name", "arr", "arr_name", "category", "cases"]]
df = df.groupby(["dept_name", "arr_name", "category"]).agg(total_cases=("cases", "sum")).reset_index()

----

In [72]:
name = input("원하는 지역? ")

원하는 지역?  파주시


In [73]:
# 첫번째 테이블
dept_region_subset = df[df["dept_name"].str.contains(name)]

,dept_name,arr_name,category,total_cases
4108,파주시,서귀포시,가공식품,147
4109,파주시,서귀포시,기타디지털/가전,7
4110,파주시,서귀포시,기타식품,9


In [75]:
category_sum = df[df["dept_name"].str.contains(name)].groupby("category")\
.agg(total_cases=("total_cases", "sum"))\
.sort_values("total_cases", ascending=False)\
.head(3)\
.reset_index()

category_sum["dept_region"] = name
top_categories = category_sum["category"].tolist()

In [77]:
category_sum = df[df["arr_name"].str.contains(name)].groupby("category")\
.agg(total_cases=("total_cases", "sum"))\
.sort_values("total_cases", ascending=False)\
.head(3)\
.reset_index()

category_sum["arr_region"] = name
top_categories = category_sum["category"].tolist()

In [96]:
def getitem(top_categories):
    chrome_options = Options()
    driver = webdriver.Chrome(service=Service('/Users/gwagdoseong/Documents/Rprogramming/chromedriver.exe'), options=chrome_options)

    max_page = 5
    result = pd.DataFrame(columns=["keyword", "item_name"])

    for word in top_categories:
        for i in range(1, max_page+1):
            URL = f'https://search.shopping.naver.com/search/all?query={word}&pagingIndex={i}&pagingSize=40&productSet=total&sort=rel'
            driver.get(URL)

            soup = BeautifulSoup(driver.page_source, 'html.parser')
            goods_list = soup.select('div.basicList_title__VfX3c')

            item_names = []
            for v in goods_list:
                item_name = v.select_one('div.basicList_title__VfX3c > a').get('title')
                item_names.append(item_name)

            df = pd.DataFrame({"keyword": [word]*len(item_names), "item_name": item_names})
            result = pd.concat([result, df], ignore_index=True)
    return result
    driver.quit()

In [97]:
results = getitem(top_categories)

In [120]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="264433")

In [121]:
cursor = mydb.cursor()
cursor.execute("SET NAMES utf8;")
cursor.execute("SET CHARACTER SET utf8;")
cursor.execute("SET character_set_connection=utf8;")
cursor.execute("SET GLOBAL local_infile = true;")

In [122]:
cursor.execute("USE LOGISTICS")

In [104]:
cursor.execute("CREATE TABLE \
logistics (dept_name VARCHAR(15)CHARACTER SET utf8,\
arr_name VARCHAR(15)CHARACTER SET utf8,\
category VARCHAR(15)CHARACTER SET utf8,\
total_cases INT) ")

In [105]:
for _, row in df.iterrows():
    cursor.execute(f"INSERT INTO logistics (dept_name, arr_name, category, total_cases) \
        VALUES ('{row['dept_name']}', '{row['arr_name']}', '{row['category']}', {row['total_cases']})")


In [94]:
name

'파주시'

In [106]:
table_name = "top_item_" + name
cursor.execute(f"CREATE TABLE {table_name} (keyword VARCHAR(15) CHARACTER SET utf8, item_name VARCHAR(65) CHARACTER SET utf8)")


In [108]:
for _, row in results.iterrows():
    cursor.execute(f"INSERT INTO {table_name} (keyword, item_name) \
        VALUES ('{row['keyword']}', '{row['item_name']}')")


- 새로운 데이터 입력하는 코드

In [115]:
sql = "INSERT INTO logistics (dept_name, arr_name, category, total_cases) VALUES (%s, %s, %s, %s)"

data = []

count = int(input("데이터를 몇 개 입력하시겠습니까? "))

for i in range(count):
    dept_name = input("출발지: ")
    arr_name = input("도착지: ")
    category = input("카테고리: ")
    total_cases = int(input("총 케이스 수: "))
    data.append((dept_name, arr_name, category, total_cases))

cursor.executemany(sql, data)


데이터를 몇 개 입력하시겠습니까?  2
출발지:  익산시
도착지:  제주시
카테고리:  123
총 케이스 수:  123
출발지:  익산시
도착지:  서귀포시
카테고리:  456
총 케이스 수:  456


- 모든 거 가져오기

In [116]:
name = input("dept_name을 입력하세요: ")

sql = "SELECT * FROM logistics WHERE dept_name = %s"
cursor.execute(sql, (name,))
result = cursor.fetchall()

result

dept_name을 입력하세요:  익산시


[('익산시', '서귀포시', '기타식품', 17),
 ('익산시', '서귀포시', '농산물', 6),
 ('익산시', '서귀포시', '생활용품', 8),
 ('익산시', '서귀포시', '수산', 55),
 ('익산시', '서귀포시', '언더웨어', 14),
 ('익산시', '제주시', 'DIY자재/용품', 14),
 ('익산시', '제주시', '기타식품', 7),
 ('익산시', '제주시', '기타패션의류', 5),
 ('익산시', '제주시', '농산물', 9),
 ('익산시', '제주시', '생활용품', 35),
 ('익산시', '제주시', '수산', 86),
 ('익산시', '제주시', '언더웨어', 53),
 ('익산시', '제주시', '위생/건강용품', 3),
 ('익산시', '제주시', '주방용품', 3),
 ('익산시', '제주시', '축산', 3),
 ('익산시', '제주시', '123', 123),
 ('익산시', '서귀포시', '456', 456)]

In [ ]:
name = input("arr_name을 입력하세요: ")

sql = "SELECT * FROM logistics WHERE dept_name = %s"
cursor.execute(sql, (name,))
result = cursor.fetchall()

result

In [112]:
table_name = "top_item_" + name
cursor.execute(f"SELECT * FROM {table_name}")
result = cursor.fetchall()

# 가져온 데이터 출력 예시
result


[('농산물', '[첫구매한정]영양쌀 (혼합/보통) 20kg'),
 ('농산물', '털보네 껍질째먹는 꿀 흠집사과'),
 ('농산물', '친환경마음밭 유기농양파 조생 양파 3kg5kg'),
 ('농산물', '수향미 골든퀸 10kg'),
 ('농산물', '행복드림 쌀 20kg'),
 ('농산물', '순풍 임산부 건조호박손 차100g+맛보기즙2포'),
 ('농산물', '굿프렌즈 새싹인삼 미니 100수 1-2년생'),
 ('농산물', '쿡네이처 국내산 황토 수미 햇 감자'),
 ('농산물', '영광농협 신동진쌀 20kg'),
 ('농산물', '애플피치 블러드애플 복숭아사과 엔부사과 피치애플 시나노골드 황옥 홍로 부사'),
 ('농산물', '[경남 하동] 청정 하동 청매실 5kg 10kg'),
 ('농산물', '유기농 지리산 장수 상황버섯 150g 선물세트'),
 ('농산물', '12만고객돌파약도라지협동조합 6년더덕청500g'),
 ('농산물', '털보네 껍질째먹는 꿀사과 흠집사과 못난이 흠과 5kg 10kg'),
 ('농산물', '니즈앤니즈 럭키푸드 해남 꿀고구마 특상 5kg'),
 ('농산물', '22년 혈당강하쌀 쌀10kg/미강분말 추가증정'),
 ('농산물', '최상급 레몬 극찬 찐후기 확인 필수입니다!'),
 ('농산물', '[22년] 영양쌀 (혼합/보통) 20kg'),
 ('농산물', '달다구니 제철 고당도 초당옥수수 그냥먹는 생옥수수 프리미엄 옥수수'),
 ('농산물', '23년 5월수확 국내산 하우스 햇 수미감자 3kg 제주감자 파근파근 햇감자 홍감자'),
 ('농산물', '국산 토종 말린 생강 건조 건생강 300g'),
 ('농산물', '최고당도 제주 한라봉 이렇게 저렴할수가!'),
 ('농산물', '[산청군농협] 지리산 맛있는 밥맛좋은쌀 백미 청정지역 산청메뚜기쌀 10kg'),
 ('농산물', '베니하루카 꿀고구마 2kg 5kg 10kg 황금 호박고구마 해남 영암'),
 ('농산물', '복숭아 하우스 신비복숭아 신선 딱딱이 납작 대극천 백도 황도 임

In [117]:
cursor.close()

True